# Import

In [1]:
import json
from jovis_model.config import Config
from run import ModelRunner

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
params = {
    "pkg": "llm",
    "task": "chat",
    "use_hf_model": True,
    "params": {
        "hf_name": "meta-llama/Meta-Llama-3-8B-Instruct"
    }
}

In [3]:
config = Config(**params)
runner = ModelRunner(config)

In [4]:
mm = runner.get_model()

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.14it/s]


In [5]:
dialogs = [
    {"role": "user", "content": "hello llama?"}
]

In [8]:
runner.run(
    job="inference",
    sample_input=dialogs
)

/home/omnious/workspace/jovis/jovis-model/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.03s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 23.68 GiB of which 77.00 MiB is free. Process 20142 has 804.00 MiB memory in use. Process 45190 has 22.82 GiB memory in use. Of the allocated memory 22.10 GiB is allocated by PyTorch, and 1.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)